**Introduction to Python for Data Science and Data Engineering**

Spark Components

  ![](https://jmp.sh/s/YzccOZReG7jdljk57jox)

  Spark Core (RDD API)-> Dataframe API -> Spark SQL/ Spark R API/ MLlib/ Structured Streaming API

# Spark Runtime Architecture
Driver / Cluster Manager / Workers / Executers
###  Driver    
- Responsible for planning and co-ordinating execution.
- Creates the SparkSession, the entry point to all spark applications.
- analyzes spark application and constructs DAG
- Schedules and distributes tasks to executors for execution
- monitors the progress of tasks and handles failures
- returns results to the client
###  Cluster Manager/Master
- Manages cluster resources and allocates them to driver
###  Workers
 - Nodes in the cluster that host ececutors.
###  Executors
- Processes on worker nodes that execute tasks assigned by the driver.
- Run on worker nodes in a spark cluster and host Tasks.
- Store intermediate and final resluts in memory or on disk.
- Interact with the driver for task co-ordination and data transfer


# The Spark DAG
 - Spark jobs are broken down in stages i.e group of tasks that can be run in parallel.
 - Computations flow in one direction through the stages
 - Stages never loop back, ensuring the job terminates
 - Stages are organized into a dependency graph for execution flow.

The Spark UI
 Visualising Spark applications

 Spark provides web user interfaces for monitoring and management including

###  Application UI
  - Per application SparkSession
  - Track Application progress and task execution
  - DAG visualization and stage details
  - Resource usage and performance metrics.

### Master UI
  - Per cluster
  - Worker node status and health and cluster-wide resource allocation
  - Shows all running applications and available resources.

# Spark Clusters in databricks

- **All purpose clusters** - interactive clusters that support notebooks , jobs, dashboards with auto termination
- **Job Cluster** - Clusters that stat when a job runs and terminate automatically upon completion, optimized for non interactive workloads.
- **SQL Warehouses** - Optimized clusters for SQL query performance with instant startup and auto-scaling to balance cost and performance.

In [64]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("MySparkSession") \
    .getOrCreate()

In [65]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import col

import math

# user defined function
def primeNumbercheck(myNum):
	if myNum==2:
		return True
	elif myNum%2==0 or myNum==1:
		return False
	x=int(math.sqrt(myNum))
	if x%2==0:
		strtno=x+1
	else:
		strtno=x
	for i in range(strtno,1,-2):
		if myNum%i==0:
			return False
	return True


In [66]:
#register the function primecheck as a
#User Defined Function (UDF) for use with Spark DataFrames
from pyspark.sql.functions import udf

@udf(returnType=BooleanType())
def primecheck(myNum):
    return primeNumbercheck(myNum)

In [74]:
#create dataframe with sample data 1000 records
df = spark.range(0,100000)

print(type(df))

#Add a new column isPrime with boolean value as output
df = df.withColumn("isPrime", primecheck(df["id"]))

<class 'pyspark.sql.dataframe.DataFrame'>


In [68]:
print(type(df))

df.filter(df.isPrime==True).show()

<class 'pyspark.sql.dataframe.DataFrame'>
+---+-------+
| id|isPrime|
+---+-------+
|  2|   true|
|  3|   true|
|  5|   true|
|  7|   true|
| 11|   true|
| 13|   true|
| 17|   true|
| 19|   true|
| 23|   true|
| 29|   true|
| 31|   true|
| 37|   true|
| 41|   true|
| 43|   true|
| 47|   true|
| 53|   true|
| 59|   true|
| 61|   true|
| 67|   true|
| 71|   true|
+---+-------+
only showing top 20 rows



In [75]:
# number of primes within 1000
df.filter(df.isPrime==True).count()

9592

In [76]:
type(df)

pyspark.sql.dataframe.DataFrame

In [11]:
#  spark.sql('select current_version()').show()
!java --version

openjdk 11.0.27 2025-04-15
OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [20]:
# install ngrok reverse proxy python wrapper
!pip install pyngrok
!pip install findspark

In [23]:
import findspark
findspark.init()
from pyngrok import ngrok
import os

In [28]:
ngrok.set_auth_token("Get your token from https://dashboard.ngrok.com/get-started/your-authtoken")

In [29]:
spark_ui_port = spark.conf.get("spark.ui.port")
ngrok_tunnel = ngrok.connect(spark_ui_port)
print(f"Spark UI URL: {ngrok_tunnel.public_url}")

Spark UI URL: https://95c5-34-16-236-147.ngrok-free.app
